# **Creating the Models**

In [8]:
tmp = "/Users/elig/tmp"

In [9]:
width = 300
height = 300
channels = 3

## **The CNN**

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(width, height, channels)))
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))

model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [34]:
# !gdown --id 1tJVQE_zckjnA5AtOujyXLXGQmxASjUzO
# !mv Weights.zip /Users/elig/tmp/Weights.zip
!unzip -o /Users/elig/tmp/Weights.zip -d ~/tmp/
weights_dir = f'./Weights'

Archive:  /Users/elig/tmp/Weights.zip
  inflating: /Users/elig/tmp/Weights/Weights.html  
  inflating: /Users/elig/tmp/Weights/Knife.h5  
  inflating: /Users/elig/tmp/Weights/Scissors.h5  
  inflating: /Users/elig/tmp/Weights/Wrench.h5  
  inflating: /Users/elig/tmp/Weights/Pliers.h5  
  inflating: /Users/elig/tmp/Weights/Gun.h5  


## **Creating a Copy for each Class**

In [35]:
classes = ['Pliers', 'Gun', 'Knife', 'Wrench', 'Scissors']

In [36]:
models_dict = dict()
for class_name in classes:
  models_dict[class_name] = tf.keras.models.clone_model(model)
  models_dict[class_name].load_weights(f'{weights_dir}/{class_name}.h5')

## **Loading Images**

In [31]:
# !gdown --id 1dYVzzSTW0KfyqqG6KU-nPAS_iZi21vpE
# !mv test_images.zip ~/tmp/test_images.zip 
!unzip -o ~/tmp/test_images.zip -d ~/tmp/

Archive:  /Users/elig/tmp/test_images.zip
   creating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03382-1.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03382-2.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03382-3.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03382-4.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03384-1.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03384-2.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03384-3.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03384-4.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03387-1.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_Scissors/P03387-2.jpg  
  inflating: /Users/elig/tmp/test_images/_Gun_Knife_Wrench_

In [37]:
test_images_dir = '~/tmp/test_images'

In [55]:
from keras.utils.image_utils import img_to_array, load_img, array_to_img

## **Test a Single Image**

In [45]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt

# test_image = '/Users/elig/tmp/test_images/No_Weapon/N1040644.jpg'
test_image = "https://eli-did-assets.s3.amazonaws.com/dataset/JPEGImage/N1040644.jpg"

predicted_classes = []
for class_name in models_dict.keys():
  #model.load_weights(f'{weights_dir}/{class_name}.h5')
  test = load_img(test_image, target_size=(300, 300))
  test = np.expand_dims(test, axis=0)
  prediction = models_dict[class_name].predict(test)
  print(f'{class_name}: {prediction}')
  if prediction >= 0.5:
    predicted_classes.append(class_name)
if len(predicted_classes) == 0:
  predicted_classes.append('No weapon')
print(predicted_classes)
img = mpimg.imread(test_image)
imgplot = plt.imshow(img)
plt.title(f'{predicted_classes}')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'https://eli-did-assets.s3.amazonaws.com/dataset/JPEGImage/N1040644.jpg'

In [58]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt

from urllib import request
from PIL import Image
from io import BytesIO


s3_url = "https://eli-did-assets.s3.amazonaws.com/dataset/JPEGImage"
image_id = "N1040644"
source_url = f'{s3_url}/{image_id}.jpg'

# test_image = '/Users/elig/tmp/test_images/No_Weapon/N1040644.jpg'
test_image = '~/tmp/N1040644.jpg'
predicted_classes = []

test = BytesIO(request.urlopen(source_url).read())
test = load_img(test, target_size=(300, 300))
test = np.expand_dims(test, axis=0)
for class_name in models_dict.keys():
  
  model.load_weights(f'{weights_dir}/{class_name}.h5')
  # np.array(Image.open(request.urlopen(url)))
  
  prediction = models_dict[class_name].predict(test)
  print(f'{class_name}: {prediction}')
  if prediction >= 0.5:
    predicted_classes.append(class_name)
if len(predicted_classes) == 0:
  predicted_classes.append('No weapon')
print(predicted_classes)
# img = mpimg.imread(test_image)
# imgplot = plt.imshow(np.array(Image.open(request.urlopen(test_image))))
# plt.title(f'{predicted_classes}')
# plt.show()

# img = mpimg.imread(test_image2)
# imgplot = plt.imshow(img)
# plt.title(f'{predicted_classes}')
# plt.show()



1/1 [==============================] - 0s 28ms/step
Pliers: [[0.]]
1/1 [==============================] - 0s 19ms/step
Gun: [[0.]]
1/1 [==============================] - 0s 19ms/step
Knife: [[0.]]
1/1 [==============================] - 0s 19ms/step
Wrench: [[0.]]
1/1 [==============================] - 0s 18ms/step
Scissors: [[0.]]
['No weapon']


## **Create Good Images Test Set**

In [ ]:
from google.colab import drive
drive.mount('/tmp/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.image_utils import img_to_array, load_img, array_to_img

In [ ]:
def predict_on_image(test_image):
  predicted_classes = []
  for class_name in models_dict.keys():
    test = load_img(test_image, target_size=(300, 300))
    test = np.expand_dims(test, axis=0)
    prediction = models_dict[class_name].predict(test)
    if prediction >= 0.5:
      predicted_classes.append(class_name)
  if len(predicted_classes) == 0:
    predicted_classes.append('No_Weapon')
  return predicted_classes


In [ ]:
target_dir = '~/tmp/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2'
source_dir = '~/tmp/drive/MyDrive/Data Science Project/Dataset_Images-New/Test_Images'

In [ ]:
import os
if not os.path.isdir(target_dir):
  os.mkdir(target_dir)

In [ ]:
import os
import shutil
for dir_name in os.listdir(source_dir):
  names = dir_name.split('_')
  if '' in names: names.remove('')
  if dir_name == 'No_Weapon':
    names = ['No_Weapon']
  dest_images_dir = f'{target_dir}/{dir_name}'
  source_images_dir = f'{source_dir}/{dir_name}'
  if not os.path.isdir(dest_images_dir):
    os.mkdir(dest_images_dir)
    print(f'{dest_images_dir}- creates')
  else:
    print(f'{dest_images_dir}- exists')
    continue
  for img in os.listdir(source_images_dir):
    predictions = predict_on_image(f'{source_images_dir}/{img}')
    if len(predictions) != len(names):
      continue
    good = True
    for name in names:
      if name not in predictions:
        good = False
        break
    if good:
      shutil.copyfile(f'{source_images_dir}/{img}', f'{dest_images_dir}/{img}')
  if len(os.listdir(dest_images_dir)) == 0:
    shutil.rmtree(dest_images_dir)




/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Wrench- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Knife_Wrench_Pliers- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Knife- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Knife_Wrench- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Pliers- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Wrench_Pliers- exists
/content/drive/MyDrive/Data Science Project/Final_Test_Images/Good_Images2/_Gun_Knife_Wrench_Scissors- creates
1/1 [==============================] - 0s 216ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 63ms/step
